In [1]:
import sys
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt

sys.path.append('python_scripts')
# local imports
from python_scripts import file_mgmt, mod_seqs, constants, ai_funcs
from python_scripts.networks.AlexNet import AlexNet
from SeqsData import SeqsData

# Author(s): Zachery Linscott and Dr. Manas Jyoti Das

In [ ]:
# check if GPU is available or else default to CPU usage.
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# One hot encode the data

print("Reading in test, train, and validation files")
test = file_mgmt.read_file(constants.TEST) # 20% of the original data
train = file_mgmt.read_file(constants.TRAIN) # 70% of the original data
validation = file_mgmt.read_file(constants.VALID) # 5% of the original data

print("Splitting the test, train and validation data into lists")
test = [line.split() for line in test]
train = [line.split() for line in train]
validation = [line.split() for line in validation]
print("\nDone")

In [ ]:
frac = 1
# for testing purposes, use only small slices of datasets
print(f"One hot encoding {frac * 100}% of the test data\n")
test_one_hot = mod_seqs.one_hot_encode(test[:int(len(test) * frac)])
# np.save(constants.ONE_HOT_TEST, test_one_hot)

print(f"One hot encoding {frac * 100}% of the train data\n")
train_one_hot = mod_seqs.one_hot_encode(train[:int(len(train) * frac)])
# np.save(constants.ONE_HOT_TRAIN, train_one_hot)

print(f"One hot encoding {frac * 100}% of the validation data\n")
valid_one_hot = mod_seqs.one_hot_encode(validation[:int(len(validation) * frac)])
# np.save(constants.ONE_HOT_VALID, valid_one_hot)

In [5]:
# create test dataset to be fed into the dataloader
np.transpose(test_one_hot)
sd_test = SeqsData(test_one_hot)

# create train dataset to be fed into dataloader
np.transpose(train_one_hot)
sd_train = SeqsData(train_one_hot)

# create train dataset to be fed into dataloader
np.transpose(valid_one_hot)
sd_valid = SeqsData(valid_one_hot)


In [6]:
# DataLoader Objects are what feed the network the data by means of tensor stacks defined in the SeqsData class.
test_dataloader = DataLoader(sd_test, batch_size=512, shuffle=True)
train_dataloader = DataLoader(sd_train, batch_size=512, shuffle=True)
valid_dataloader = DataLoader(sd_valid, batch_size=512, shuffle=True)

In [7]:
# transform_conf = transforms.Compose([
#     # transforms.Resize((224,224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,),(0.3081,))
# ])

In [8]:
# Instantiating the model and assigning an optimizer to the model and creating a loss function
model = AlexNet()
model.load_state_dict(torch.load("saved_models/calexnet8.pth"))
model = model.to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [9]:
if __name__=='__main__':
    seed=42
    EPOCHS = 10 # run for 10 epochs at least
    
    for epoch in range(1, EPOCHS + 1):
        ai_funcs.train(model, device, train_dataloader, optimizer,epoch, loss_fn)
        ai_funcs.test(model,device,test_dataloader)
        ai_funcs.validation(model, device, valid_dataloader)
        torch.save(model.state_dict(), constants.ALEXNET + f"{epoch}" + ".pth") # save model for each epoch

In [ ]:
# model = torch.load("zachnet.pth")
# print(model)
# model.eval()